# Import libraries 

In [1]:
import numpy as np
import pandas as pd
import csv
from scipy import sparse
from fractions import Fraction

# Display float values as decimals instead of scientific notation 
pd.options.display.float_format = '{:f}'.format

# Set dataframe parameters
import sys
from IPython.display import display
np.set_printoptions(threshold=sys.maxsize, suppress=True)

# Load dataset

In [2]:
df = pd.read_csv("chicago-taxi-rides.csv")
df.dropna(inplace=True)
df = df.astype(int)
df.reset_index(inplace=True, drop=True)
df = df.sort_values(by=['pickup_community_area', 'dropoff_community_area'], ascending=True)
df

,pickup_community_area,dropoff_community_area,trips
4221,1,1,110697
4222,1,2,40535
927,1,3,25228
483,1,4,8677
5057,1,5,3347
...,...,...,...
5148,77,73,62
2710,77,74,6
4845,77,75,10
2870,77,76,35950


# Build Traffic Matrix

In [3]:
# Set parameters
N = 77
beta = 0.85 

# Create ivj matrix, store entries as floats for division and normalization
col = df["pickup_community_area"]
row = df["dropoff_community_area"]
value = df["trips"]
mat = sparse.coo_matrix((value, (row-1, col-1)), shape=(N, N)).toarray().astype(float)
display(pd.DataFrame(mat))

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
0,110697.000000,28110.000000,41575.000000,9768.000000,5228.000000,63661.000000,21553.000000,70134.000000,28.000000,217.000000,...,10.000000,10.000000,6.000000,8.000000,9.000000,1.000000,0.000000,2.000000,35848.000000,69417.000000
1,40535.000000,169484.000000,22064.000000,28753.000000,5752.000000,25556.000000,10921.000000,37972.000000,37.000000,628.000000,...,11.000000,16.000000,7.000000,5.000000,2.000000,2.000000,1.000000,3.000000,30926.000000,44670.000000
2,25228.000000,12978.000000,378179.000000,43074.000000,34371.000000,440033.000000,105724.000000,316526.000000,11.000000,254.000000,...,173.000000,79.000000,14.000000,43.000000,4.000000,22.000000,1.000000,13.000000,75845.000000,122363.000000
3,8677.000000,17694.000000,63660.000000,100298.000000,30060.000000,110674.000000,37902.000000,110122.000000,28.000000,236.000000,...,7.000000,8.000000,4.000000,4.000000,2.000000,0.000000,0.000000,0.000000,52146.000000,48455.000000
4,3347.000000,3473.000000,38167.000000,21596.000000,84667.000000,207812.000000,77115.000000,167135.000000,18.000000,199.000000,...,3.000000,17.000000,3.000000,3.000000,2.000000,2.000000,3.000000,2.000000,70526.000000,14384.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,8.000000,10.000000,40.000000,8.000000,4.000000,100.000000,71.000000,712.000000,0.000000,2.000000,...,35.000000,229.000000,10.000000,70.000000,35.000000,938.000000,2.000000,43.000000,211.000000,62.000000
73,7.000000,0.000000,30.000000,1.000000,4.000000,166.000000,62.000000,377.000000,0.000000,1.000000,...,0.000000,2.000000,2.000000,3.000000,46.000000,3.000000,198.000000,25.000000,293.000000,6.000000
74,5.000000,6.000000,16.000000,9.000000,22.000000,173.000000,248.000000,643.000000,0.000000,1.000000,...,21.000000,38.000000,11.000000,17.000000,55.000000,40.000000,19.000000,635.000000,458.000000,10.000000
75,13054.000000,10569.000000,39343.000000,19050.000000,27593.000000,159967.000000,118746.000000,1119023.000000,491.000000,7284.000000,...,77.000000,77.000000,93.000000,104.000000,117.000000,52.000000,17.000000,53.000000,590584.000000,35950.000000


# Implement TrafficRank Algorithm

In [4]:
# Normalize matrix entries by column sums, i.e. out-degrees
mat /= mat.sum(axis = 0)
display(pd.DataFrame(mat))

init = np.full((N, N), Fraction(1, N))
r_new = np.full((N, 1), Fraction(1, N))
display(pd.DataFrame(init))

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
0,0.314364,0.082866,0.025775,0.020971,0.008201,0.009504,0.003997,0.002220,0.011006,0.004620,...,0.001163,0.000731,0.001080,0.001203,0.004487,0.000289,0.000000,0.001072,0.007654,0.066751
1,0.115114,0.499626,0.013679,0.061731,0.009023,0.003815,0.002025,0.001202,0.014544,0.013369,...,0.001279,0.001169,0.001261,0.000752,0.000997,0.000578,0.002278,0.001608,0.006603,0.042955
2,0.071644,0.038258,0.234461,0.092477,0.053918,0.065692,0.019605,0.010018,0.004324,0.005407,...,0.020119,0.005774,0.002521,0.006465,0.001994,0.006357,0.002278,0.006967,0.016193,0.117664
3,0.024641,0.052161,0.039468,0.215333,0.047155,0.016522,0.007028,0.003485,0.011006,0.005024,...,0.000814,0.000585,0.000720,0.000601,0.000997,0.000000,0.000000,0.000000,0.011133,0.046594
4,0.009505,0.010238,0.023663,0.046365,0.132817,0.031024,0.014300,0.005290,0.007075,0.004236,...,0.000349,0.001242,0.000540,0.000451,0.000997,0.000578,0.006834,0.001072,0.015058,0.013832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,0.000023,0.000029,0.000025,0.000017,0.000006,0.000015,0.000013,0.000023,0.000000,0.000043,...,0.004070,0.016736,0.001801,0.010525,0.017448,0.271020,0.004556,0.023044,0.000045,0.000060
73,0.000020,0.000000,0.000019,0.000002,0.000006,0.000025,0.000011,0.000012,0.000000,0.000021,...,0.000000,0.000146,0.000360,0.000451,0.022931,0.000867,0.451025,0.013398,0.000063,0.000006
74,0.000014,0.000018,0.000010,0.000019,0.000035,0.000026,0.000046,0.000020,0.000000,0.000021,...,0.002442,0.002777,0.001981,0.002556,0.027418,0.011557,0.043280,0.340300,0.000098,0.000010
75,0.037072,0.031157,0.024392,0.040899,0.043285,0.023881,0.022019,0.035417,0.193003,0.155068,...,0.008955,0.005627,0.016748,0.015637,0.058325,0.015025,0.038724,0.028403,0.126092,0.034569


,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
0,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,...,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77
1,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,...,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77
2,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,...,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77
3,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,...,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77
4,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,...,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,...,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77
73,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,...,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77
74,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,...,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77
75,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,...,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77,1/77


##### Follow a link with probability beta, jump to random page with probability (1-beta)

In [5]:
A = (beta * mat) + ((1 - beta) * init) 

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
0,0.269157,0.072384,0.023857,0.019774,0.008919,0.010026,0.005345,0.003835,0.011303,0.005875,...,0.002937,0.002569,0.002866,0.002970,0.005762,0.002194,0.001948,0.002859,0.008454,0.058687
1,0.099795,0.426630,0.013575,0.054419,0.009618,0.005191,0.003669,0.002970,0.014310,0.013312,...,0.003035,0.002942,0.003020,0.002587,0.002796,0.002439,0.003884,0.003315,0.007560,0.038459
2,0.062845,0.034467,0.201240,0.080553,0.047778,0.057786,0.018612,0.010463,0.005623,0.006544,...,0.019049,0.006856,0.004091,0.007443,0.003643,0.007351,0.003884,0.007870,0.015712,0.101962
3,0.022893,0.046285,0.035496,0.184981,0.042030,0.015992,0.007922,0.004911,0.011303,0.006219,...,0.002640,0.002445,0.002560,0.002459,0.002796,0.001948,0.001948,0.001948,0.011411,0.041553
4,0.010027,0.010650,0.022061,0.041358,0.114843,0.028318,0.014103,0.006444,0.007962,0.005549,...,0.002245,0.003004,0.002407,0.002331,0.002796,0.002439,0.007757,0.002859,0.014747,0.013705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,0.001967,0.001973,0.001969,0.001963,0.001953,0.001961,0.001959,0.001967,0.001948,0.001984,...,0.005408,0.016174,0.003479,0.010894,0.016779,0.232315,0.005820,0.021535,0.001986,0.001999
73,0.001965,0.001948,0.001964,0.001950,0.001953,0.001969,0.001958,0.001958,0.001948,0.001966,...,0.001948,0.002072,0.002254,0.002331,0.021440,0.002685,0.385319,0.013336,0.002001,0.001953
74,0.001960,0.001963,0.001956,0.001964,0.001977,0.001970,0.001987,0.001965,0.001948,0.001966,...,0.004024,0.004309,0.003632,0.004121,0.025253,0.011772,0.038736,0.291203,0.002031,0.001956
75,0.033459,0.028431,0.022681,0.036712,0.038740,0.022247,0.020664,0.032052,0.166001,0.133756,...,0.009559,0.006731,0.016184,0.015239,0.051524,0.014719,0.034864,0.026091,0.109126,0.031332


### Instruction: Using your formulation of the TrafficRank algorithm, calculate the rankings of the Chicago community areas after 2, 4, 8, … 64 iterations of the algorithm. 

In [6]:
check_iter_result = [2**i for i in range(1,7)]
rank_per_iter = []

In [7]:
# Run 100 iterations of the algorithm
for i in range(100):
    r_new = A @ r_new
    # Store rankings after 2, 4, 8, ... 64 iterations
    if i+1 in check_iter_result:
        rank_per_iter.append(r_new.reshape(77,))

In [8]:
ranks = np.array(rank_per_iter).T
headers = ['Iter ' + str(i) for i in check_iter_result]
ranks_df = pd.DataFrame(ranks, columns=headers)
display(ranks_df)

,Iter 2,Iter 4,Iter 8,Iter 16,Iter 32,Iter 64
0,0.009410,0.009185,0.009230,0.009272,0.009274,0.009274
1,0.013435,0.012296,0.011801,0.011804,0.011807,0.011807
2,0.018240,0.019509,0.020332,0.020495,0.020502,0.020502
3,0.010888,0.010896,0.011074,0.011138,0.011141,0.011141
4,0.009549,0.010430,0.010935,0.011025,0.011028,0.011028
...,...,...,...,...,...,...
72,0.005026,0.003943,0.003595,0.003549,0.003548,0.003548
73,0.005178,0.003782,0.003478,0.003464,0.003463,0.003463
74,0.005367,0.004014,0.003665,0.003633,0.003633,0.003633
75,0.031307,0.031244,0.031331,0.031402,0.031408,0.031408


In [9]:
# Check columns sum to 1
ranks_df.sum(axis=0)

Iter 2    1.000000
Iter 4    1.000000
Iter 8    1.000000
Iter 16   1.000000
Iter 32   1.000000
Iter 64   1.000000
dtype: object

# View Results

In [10]:
city_codes = pd.Series(range(0,77))
iter_64 = pd.Series(ranks_df['Iter 64'])
results_64 = pd.DataFrame(dict(AreaCodes=city_codes, Rank=iter_64))
# Sort by rank
results_64.sort_values(by=['Rank'], ascending=False, inplace=True)
print(results_64.head(10))
print(results_64.tail(10))

    AreaCodes     Rank
7           7 0.178175
31         31 0.121137
27         27 0.069738
5           5 0.055061
6           6 0.044328
23         23 0.037468
75         75 0.031408
32         32 0.023680
2           2 0.020502
55         55 0.019059
    AreaCodes     Rank
56         56 0.003389
19         19 0.003348
35         35 0.003335
25         25 0.003310
8           8 0.003128
17         17 0.003001
61         61 0.002945
51         51 0.002890
54         54 0.002877
46         46 0.002819


In [11]:
r_new = r_new.flatten()
results = pd.Series(r_new)
conv_results = pd.DataFrame(dict(AreaCodes=city_codes, TrafficRank=results))
# Sort by rank
conv_results.sort_values(by=['TrafficRank'], ascending=False, inplace=True)
print(conv_results.head(10))
print(conv_results.tail(10))

    AreaCodes TrafficRank
7           7    0.178175
31         31    0.121137
27         27    0.069738
5           5    0.055061
6           6    0.044328
23         23    0.037468
75         75    0.031408
32         32    0.023680
2           2    0.020502
55         55    0.019059
    AreaCodes TrafficRank
56         56    0.003389
19         19    0.003348
35         35    0.003335
25         25    0.003310
8           8    0.003128
17         17    0.003001
61         61    0.002945
51         51    0.002890
54         54    0.002877
46         46    0.002819
